## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [4]:
import glob

import numpy as np
import scipy as sp
import scipy.io
import scipy.signal


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estim0tes = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)


def BandpassFilter(signal, fs, pass_band=(40/60, 240/60)):
    """
    Apply a Butterworth bandpass filter to the signal.

    Args:
        signal (array): The input signal to filter.
        pass_band (tuple): The lower and upper frequencies for the bandpass filter.
        fs (int): The sampling frequency of the signal.

    Returns:
        array: The bandpass filtered signal.
    """
    b, a = sp.signal.butter(3, pass_band, btype='bandpass', fs=fs)
    return sp.signal.filtfilt(b, a, signal)

def FourierTransform(signal, fs):
    """
    Perform a Fourier Transform on the signal.

    Args:
        signal (array): The input signal to transform.
        fs (int): The sampling frequency of the signal.

    Returns:
        tuple: Frequencies and the corresponding magnitudes of the signal.
    """
    freqs = np.fft.rfftfreq(2 * len(signal), 1 / fs)
    fft = np.abs(np.fft.rfft(signal, 2 * len(signal)))
    return freqs, fft

def CreateFeatures(ppg, accx, accy, accz, fs):
    """
    Create features from PPG and accelerometer data.

    Args:
        ppg (array): The PPG signal.
        accx (array): The x-axis accelerometer signal.
        accy (array): The y-axis accelerometer signal.
        accz (array): The z-axis accelerometer signal.
        fs (int): The sampling frequency.
    
    Returns:
        tuple: Two dictionaries containing FFT and peak information for the accelerometer and PPG signals.
            acc_info (dict): Contains the FFT data, frequencies, peaks, peak frequencies, and peak values for the accelerometer.
            ppg_info (dict): Contains the FFT data, frequencies, peaks, peak frequencies, and peak values for the PPG.
    """
    # Frequency range for filtering (in Hz)
    min_freq = 40 / 60
    max_freq = 240 / 60
    
    # Combine accelerometer signals and perform Fourier Transform
    acc = np.sqrt(accx**2 + accy**2 + accz**2)
    acc_freqs, acc_fft = FourierTransform(acc, fs)
    acc_fft[(acc_freqs <= min_freq) | (acc_freqs >= max_freq)] = 0.0
    
    # Find acc peaks
    acc_peaks, acc_peaks_f  = FindPeaks(acc_fft, acc_freqs)
    
    # max freq for acc
    acc_peak_freq = acc_freqs[np.argmax(acc_fft)]
    
    # Fourier Transform for PPG
    ppg_freqs, ppg_fft = FourierTransform(ppg, fs)
    ppg_fft[(ppg_freqs <= min_freq) | (ppg_freqs >= max_freq)] = 0.0
    
    # Find ppg peaks
    ppg_heigh = 2000
    ppg_peaks, ppg_peaks_f = FindPeaks(ppg_fft, ppg_freqs, ppg_heigh)
    
    # max freq for ppg
    ppg_peak_freq = ppg_freqs[np.argmax(ppg_fft)]
    
    
    acc_info = {
        'fft': acc_fft,
        'freqs': acc_freqs,
        'peaks': acc_peaks,
        'peaks_f': acc_peaks_f,
        'peak_freq': acc_peak_freq
    }

    ppg_info = {
        'fft': ppg_fft,
        'freqs': ppg_freqs,
        'peaks': ppg_peaks,
        'peaks_f': ppg_peaks_f,
        'peak_freq': ppg_peak_freq
    }
    return acc_info, ppg_info


def FindPeaks(fft, freqs, height=None):
    """
    Identify peaks in the FFT magnitude spectrum and their corresponding frequencies.
    
    Parameters:
    fft (array): FFT magnitude spectrum.
    freqs (array): Frequencies corresponding to the FFT spectrum.
    height (float, optional): Minimum height of peaks to be identified. Default is None.
    
    Returns:
    tuple:
        fft_peaks (array): Indices of the identified peaks in the FFT spectrum.
        fft_peaks_f (array): Frequencies corresponding to the identified peaks.
    """
    # Find peaks in the FFT magnitude spectrum
    fft_peaks = sp.signal.find_peaks(fft, height=height)[0]
    
    # Get the frequencies corresponding to the identified peaks
    fft_peaks_f = freqs[fft_peaks]
    
    return fft_peaks, fft_peaks_f

def EstimateHeartRateWithConfidence(ppg, accx, accy, accz, window_length_s, window_shift_s, fs):
    """
    Estimate the heart rate and calculate the confidence of the estimation.

    Args:
        ppg (array): The PPG signal.
        acc (array): The combined accelerometer signal.
        window_length_s (int): The length of each analysis window in seconds.
        window_shift_s (int): The shift between successive windows in seconds.
        fs (int): The sampling frequency.

    Returns:
        tuple: The estimated heart rate (in bpm) and the confidence of the estimation.
    """

    window_length = window_length_s * fs
    window_shift = window_shift_s * fs
    
    estimated_bpm = []
    confidence_scores = []
    
    previous_estimation = 40 / 60
    
    # Loop through the signal with defined window length and shift
    for i in range(0, len(ppg) - window_length, window_shift):
        k = 1
        ppg_window = ppg[i:i + window_length]
        
        # Create features from the windowed signals
        acc_info, ppg_info = CreateFeatures(ppg_window, accx, accy, accz, fs)
        
        # Ensure the peak frequency is not influenced by the accelerometer
        while np.abs(acc_info['peak_freq'] - ppg_info['peak_freq']) <= 0.2 and k <= 2:
            k += 1
            ppg_info['peak_freq'] = ppg_info['freqs'][np.argsort(ppg_info['fft'], axis=0)[-k]]
            acc_info['peak_freq'] = acc_info['freqs'][np.argsort(acc_info['fft'], axis=0)[-k]]
        
        estimated_bpm_freq = ppg_info['peak_freq']
        previous_estimation = estimated_bpm_freq
        
        # Append the estimated BPM and the confidence score
        estimated_bpm.append(estimated_bpm_freq * 60)
        frequency_window_range_hz = 30 / 60
        frequency_window = (ppg_info['freqs'] > estimated_bpm_freq - frequency_window_range_hz) & (ppg_info['freqs'] < estimated_bpm_freq + frequency_window_range_hz)
        conf = np.sum(ppg_info['fft'][frequency_window]) / np.sum(ppg_info['fft'])
        confidence_scores.append(conf)

    return estimated_bpm, confidence_scores



def RunPulseRateAlgorithm(data_fl, ref_fl):
    """
    Run the pulse rate algorithm on the given data file and reference file.

    Args:
        data_fl (str): Path to the data file containing the signals.
        ref_fl (str): Path to the reference file containing the ground truth heart rates.

    Returns:
        tuple: Per-estimate mean absolute error and confidence as numpy arrays.
    """
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    bpm = sp.io.loadmat(ref_fl)['BPM0']
    
    fs = 125
    
    # Bandpass Filter the signals
    ppg = BandpassFilter(ppg, fs)
    accx = BandpassFilter(accx, fs)
    accy = BandpassFilter(accy, fs)
    accz = BandpassFilter(accz, fs)
    
    # Compute pulse rate estimates and estimation confidence
    estimated_bpm, confidence_scores = EstimateHeartRateWithConfidence(ppg, accx, accy, accz, 8, 2, fs)

    # Calculate the mean absolute error
    errors = np.abs(np.diag(np.subtract(bpm, estimated_bpm)))
    
    return errors, confidence_scores

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

Your write-up goes here...

## Code Description

### How to Run the Code

To run the code and use the algorithm:

1. **Dataset Preparation**: Ensure the Troika dataset is correctly downloaded and stored in the specified directory (`./datasets/troika/training_data`). This dataset consists of pairs of `.mat` files: `DATA_*.mat` containing signal data and `REF_*.mat` containing reference heart rates.

2. **Functions Provided**:
   - `LoadTroikaDataset()`: Retrieves the filenames of the dataset files.
   - `LoadTroikaDataFile(data_fl)`: Loads and extracts signals (PPG, accelerometer data) from a specific Troika `.mat` file.
   - `RunPulseRateAlgorithm(data_fl, ref_fl)`: Runs the pulse rate estimation algorithm on a given data file and corresponding reference file.
   - Various signal processing functions (`BandpassFilter`, `FourierTransform`, `CreateFeatures`, etc.) used within the algorithm.

3. **Execution**: 
   - Import necessary libraries (`numpy`, `scipy`).
   - Ensure all functions (`RunPulseRateAlgorithm`, signal processing functions) are accessible.
   - Call `Evaluate()` function to compute aggregate error metrics over the entire Troika dataset.

## Data Description

### Dataset Used

The Troika dataset used consists of PPG signals along with accelerometer data from wrist-worn devices. These signals are collected during various activities to estimate heart rates. Each `.mat` file pair (`DATA_*.mat` and `REF_*.mat`) provides:
- PPG signal (photoplethysmogram).
- Three-axis accelerometer signals (x, y, z).
- Reference heart rate (BPM0) derived from independent measurements.

### Shortcomings

**Shortcomings**:
- Lack of detailed contextual information (e.g., activity type, subject's health status).
- Limited diversity in demographic factors such as age, fitness level, and health conditions.

### Improvements

**Improvements**:
- To build a more complete dataset, additional data capturing diverse activities, demographics, and health conditions would be beneficial. This would enhance algorithm robustness and generalizability.

## Algorithm Description

### How the Algorithm Works

**How the Algorithm Works**:
1. **Data Preprocessing**: Signals are bandpass filtered to focus on frequency components relevant for heart rate estimation.
2. **Feature Extraction**: Features are extracted using Fourier Transform to identify dominant frequency peaks in both PPG and accelerometer signals.
3. **Heart Rate Estimation**: 
   - By analyzing the peak frequencies and their magnitudes, the algorithm estimates the heart rate.
   - Confidence in each estimation is computed based on the relative magnitude of the dominant peaks.

### Specific Aspects of Physiology

**Physiological Aspects Taken Advantage of**:
- PPG signals reflect changes in blood volume in the wrist, synchronized with heartbeats.
- Accelerometer signals provide motion context, aiding in distinguishing true heart rate-related pulsations from motion artifacts.

### Algorithm Outputs

**Algorithm Outputs**:
- Estimated heart rate (in beats per minute, BPM) for each analysis window.
- Confidence score indicating the reliability of each heart rate estimate.

### Caveats on Algorithm Outputs

**Caveats on Algorithm Outputs**:
- Motion artifacts or irregular physiological responses may affect estimation accuracy.
- Confidence scores are relative and may vary based on signal quality and activity type.

### Common Failure Modes

**Common Failure Modes**:
- Insufficient motion filtering leading to inaccurate peak identification.
- Inaccurate heart rate estimates due to weak PPG signals or excessive noise.

## Algorithm Performance

### Performance Evaluation

**Performance Evaluation**:
- **Error Metric**: Mean Absolute Error (MAE) at 90% availability (i.e., considering the best estimates with higher confidence).
- **Cross-Validation**: The algorithm evaluates performance across different activities and subjects within the Troika dataset.
- **Metrics Optimized For**: Optimized to minimize MAE, particularly focusing on accurate heart rate estimation during various activities and conditions.

### Generalizability

**Generalizability**:
- Performance metrics are specific to the Troika dataset and may vary on different datasets due to varying signal quality and characteristics.
- Acknowledgment of limitations in generalizing results to broader populations or activities not represented in the dataset.

-----
### Next Steps
You will now go to **Test Your Algorithm** to apply a unit test to confirm that your algorithm met the success criteria. 